In [4]:
## run matlab from python

In [2]:
import matlab.engine
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
cd "D:\git_repo\NET\timeSeriesViewer\dataRecordingObjects\"

In [ ]:
eng = matlab.engine.start_matlab() #the engine is set to the current path

In [ ]:
# eng.pwd() # make sure good path

In [ ]:
bin_fullpath = r'F:\experiment1_109.raw.bin' #NOTE: meta file shoud be with the bin file at the same folder

In [ ]:
bin_obj = eng.binaryRecording(bin_fullpath) #NOTE- if faile - make sure you created eng in the rigth path

In [ ]:
startTime_ms = matlab.double([100]) #crate a matlab array (numbers should be doubles)

In [ ]:
channels = matlab.double(range(1,2))

In [ ]:
matlab_mat_data = eng.getData(bin_obj, channels, startTime_ms, 3600000.0)

In [ ]:
py_mat = np.array(matlab_mat_data) #convert the result to python object

In [ ]:
py_mat.shape

In [ ]:
# 200000 * (1/20000) 

In [ ]:
# dim explain
# py_mat[x][y][z]
# x- channel (acording to the order given to getData)
# y- start_time (acording to the order given to getData) 
# z- voltage (acording to the window given to getData) 

In [ ]:
pc_pymat = py_mat[0][0]

In [ ]:
plt.plot(pc_pymat)

In [ ]:
# plt.figure(figsize=(15,5))
plt.plot(pc_pymat[10000:20000])

In [ ]:
stochastic_process = [0]
white_noise = []

In [ ]:
stochastic_process = [0]
for i in range(10000):
    stochastic_process.append(stochastic_process[-1]+np.random.uniform(low=-1,high=1))
#     stochastic_process.append(stochastic_process[-1]+np.random.randn())

In [ ]:
for i in range(10000):
    white_noise.append(np.random.uniform(low=-1,high=1))

In [ ]:
plt.plot(stochastic_process)

In [ ]:
from scipy import signal
import matplotlib.pyplot as plt

In [ ]:
pc_pymat.shape

In [ ]:
1200000 / 20000

In [ ]:
sin = np.sin(2*np.pi * np.linspace(0,60,1200000) * 1000 ) * 10

In [ ]:
np.sin

In [ ]:
import librosa

In [ ]:
w,sr =librosa.load(r"C:\Users\ofir1\Desktop\sin.wav",sr=None)

In [ ]:
#f, t, Zxx = signal.stft(x, fs, nperseg=1000)
# x = the signal (vector of samples) ; fs = sample rate ; nperseg = window (num of samples)
f, t, Zxx = signal.stft(pc_pymat, 20000, nperseg=200000, noverlap = 180000)


In [ ]:
import pickle

In [ ]:
# save after sftf- signal.stft(pc_pymat, 20000, nperseg=200000, noverlap = 180000)
filename = 'Zxx'
outfile = open(filename,'wb')
pickle.dump(Zxx,outfile)
outfile.close()
filename = 't'
outfile = open(filename,'wb')
pickle.dump(t,outfile)
outfile.close()
filename = 'f'
outfile = open(filename,'wb')
pickle.dump(f,outfile)
outfile.close()

In [ ]:
# save pure matlab data - startTime_ms = matlab.double([100]) ; channel = 1, window = 3600000
filename = 'mat_data'
outfile = open(filename,'wb')
pickle.dump(pc_pymat,outfile)
outfile.close()

In [ ]:
Zxx.shape

In [ ]:
len(t)

In [ ]:
plt.pcolormesh(t[:2000], f[:500], np.log(np.abs(Zxx[:500,:2000])), shading='gouraud')
plt.title('STFT Magnitude')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time')
plt.show()

In [ ]:
plt.pcolormesh(t[:2000], f[:50], np.log(np.abs(Zxx[:50,:2000])), shading='gouraud')
plt.title('STFT Magnitude')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time')
plt.show()

In [ ]:
from scipy.signal import butter,filtfilt
order = 100
# Filter requirements.
T = 5.0         # Sample Period
fs = 20000.0       # sample rate, Hz
cutoff = 2      # desired cutoff frequency of the filter, Hz ,slightly higher than actual 1.2 Hz
nyq = 0.5 * fs  # Nyquist Frequency

normal_cutoff = cutoff / nyq
b, a = butter(order, normal_cutoff, btype='low', analog=False)
y = filtfilt(b, a, pc_pymat)

In [ ]:
f, t, Zxx = signal.stft(pc_pymat, 20000, nperseg=30000)
plt.pcolormesh(t, f[:], np.log(np.abs(Zxx[:,])), shading='gouraud')
plt.title('STFT Magnitude')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time')
plt.show()

In [ ]:
np.abs(Zxx[:,]).shape

In [ ]:
# analyze

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
X.shape

In [ ]:
X = np.abs(Zxx[:500,]) #data matrix- until 50Hz
X_t = np.transpose(X)
kmeans = KMeans(n_clusters=3, random_state=0).fit(X_t) 

In [ ]:
lables = kmeans.labels_ 

In [ ]:
# lables

In [ ]:
indxes = np.where(lables == 2)

In [ ]:
sub_X = X_t[indxes[0],:]

In [ ]:
# lables

In [ ]:
sub_X.shape[0]

In [ ]:
# Zxx.shape

In [ ]:
# min(sub_X[0])

In [ ]:
plt.figure(figsize=(sub_X.shape[0] / 100 ,5))
plt.pcolormesh(t[:sub_X.shape[0]], f[:500], np.log(np.transpose(sub_X)), shading='gouraud')
plt.title('STFT Magnitude')
plt.ylabel('Frequency [Hz]')
# plt.xlabel('Time')
plt.show()

In [ ]:
plt.pcolormesh(t[:223], f[:50], (np.transpose(sub_X)), shading='gouraud')
plt.title('STFT Magnitude')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time')
plt.show()

In [ ]:
# >>> kmeans.labels_
# array([1, 1, 1, 0, 0, 0], dtype=int32)
# >>> kmeans.predict([[0, 0], [12, 3]])
# array([1, 0], dtype=int32)
# >>> kmeans.cluster_centers_
# array([[10.,  2.],
#        [ 1.,  2.]])